In [1]:
import cv2 as cv
import livestreamer
import streamlink

In [2]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import Video
import ipywidgets as widgets
import pandas as pd
import seaborn as sns
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

WHITE = (255,255,255)

class Rectangle:
    def from_contour(self, contour, mode='avg'):
        if mode == 'bound':
            x,y,w,h = cv.boundingRect(contour)
            self.x1 = x
            self.y1 = y
            self.x2 = x+w
            self.y2 = y+h
        elif mode == 'avg':
            assert len(contour) == 4
            xs = np.sort(contour[:,0,0]).view(np.ndarray)
            ys = np.sort(contour[:,0,1]).view(np.ndarray)
            self.x1 = xs[:2].mean().astype(int)
            self.x2 = xs[2:].mean().astype(int)
            self.y1 = ys[:2].mean().astype(int)
            self.y2 = ys[2:].mean().astype(int)
        else:
            raise Exception('Unknown Mode',mode)
        return self
    
    @property
    def width(self): return self.x2-self.x1

    @property
    def height(self): return self.y2-self.y1
    
    @property
    def perimeter(self): return self.width*2+self.height*2
    
    @property
    def area(self): return self.width*self.height
    
    @property
    def x(self): return self.x1
    
    @property
    def y(self): return self.y1
    
    @property
    def cx(self): return self.x1 + self.width//2
    
    @property
    def cy(self): return self.y1 + self.height//2
    
    @property
    def tl(self): return self.x1, self.y1
    
    @property
    def br(self): return self.x2, self.y2
    
    @property
    def bl(self): return self.x1, self.y2
    
    @property
    def tr(self): return self.x2, self.y1
    
    def __repr__(self):
        return 'Rect({}, {}, {}, {})'.format(self.x, self.y, self.width, self.height)

class Contour(np.ndarray):
    def __new__(self, input_array):
        obj = np.asarray(input_array).view(self)
        obj.next = None
        obj.prev = None
        obj.child = None
        obj.parent = None
        return obj
    
    def area(self):
        return cv.contourArea(self)
    
    def arcLength(self, closed=True):
        return cv.arcLength(self, closed)
    
    def approx_poly(self, percent=0.1, closed=True):
        epsilon = percent*self.arcLength(closed)
        return cv.approxPolyDP(self, epsilon, closed).view(Contour)
    
    def convex_hull(self, clockwise=True, returnPoints=True):
        return cv.convexHull(self, clockwise, returnPoints).view(Contour)
    
    def bounding_rect(self):
        return cv.boundingRect(self)

class Frame(np.ndarray):
    def __new__(self, input_array):
        obj = np.asarray(input_array).view(self)
        return obj
    
    @classmethod
    def from_file(self, filename):
        return cv.imread(filename).view(Frame)
    
    def gaussian_blur(self,*args,**kwargs):
        return cv.GaussianBlur(self,*args,**kwargs).view(Frame)
    
    def median_blur(self,*args,**kwargs):
        return cv.medianBlur(self,*args,**kwargs).view(Frame)
    
    def bilateral_filter(self, d=9, sigmaColor=75, sigmaSpace=None):
        if sigmaSpace is None:
            sigmaSpace = sigmaColor
        return cv.bilateralFilter(self, d, sigmaColor, sigmaSpace).view(Frame)
    
    def thresh(self, low, high, typ):
        typ = getattr(cv, 'THRESH_'+typ.upper())
        ret,thresh = cv.threshold(self,low,high,typ)
        return thresh.view(Frame)
    
    def canny(self, minVal, maxVal=None):
        maxVal = maxVal if maxVal is not None else minVal
        return cv.Canny(self, minVal, maxVal).view(Frame)
    
    def invert(self):
        return cv.bitwise_not(self).view(Frame)
    
    def gray(self):
        return cv.cvtColor(self, cv.COLOR_BGR2GRAY).view(Frame)
    
    def hsv_range(self, low, high):
        return cv.inRange(self.as_hsv(), low, high).view(Frame)
    
    def cvt_color(self, color):
        color = getattr(cv, 'COLOR_'+color.upper())
        return cv.cvtColor(self, color).view(Frame)
    
    def as_hsv(self):
        return self.cvt_color('bgr2hsv')
    
    def apply_mask(self, mask):
        mask = mask>0
        masked = np.zeros_like(self,np.uint8)
        masked[mask] = self[mask]
        return masked.view(Frame)
    
    def erode(self, size=3, shape='rect', anchor=(-1,-1), iterations=1, kernel=None):
        if kernel is None:
            if type(size) != tuple:
                size = (size, size)
            # rect, cross, ellipse
            shape = getattr(cv, 'MORPH_'+shape.upper())
            kernel = cv.getStructuringElement(shape, size)
        return cv.erode(self, kernel=kernel, anchor=anchor, iterations=iterations).view(Frame)
    
    def dilate(self, size=3, shape='rect', anchor=(-1,-1), iterations=1, kernel=None):
        if kernel is None:
            if type(size) != tuple:
                size = (size, size)
            # rect, cross, ellipse
            shape = getattr(cv, 'MORPH_'+shape.upper())
            kernel = cv.getStructuringElement(shape, size)
        return cv.dilate(self, kernel=kernel, anchor=anchor, iterations=iterations).view(Frame)
        
    def hough_lines(self, rho=1, theta=2, threshold=0, minLineLength=20, maxLineGap=1):
        return cv.HoughLinesP(self, 
                              rho=rho, 
                              theta=np.pi/theta, 
                              threshold=threshold, 
                              minLineLength=minLineLength, 
                              maxLineGap=maxLineGap,
                             )
    
    def draw_lines(self, lines, color=(0,255,0), thickness=1, replace=False):
        img = self if replace else self.copy()
        for line in lines[:,0]:
            cv.line(img, line[:2], line[2:], color, thickness)
        return img
    
    def find_contours(self, mode='tree', method='simple'):
        # external, list, ccomp, tree, floodfill
        mode = getattr(cv,'RETR_'+mode.upper())
        # none, simple, tc89_l1, tc89_kcos
        method = getattr(cv, 'CHAIN_APPROX_'+method.upper())
        
        raw_contours, hiearchy = cv.findContours(self, mode, method)
        contours = [Contour(contour) for contour in raw_contours]
        for c,(nxt,prev,child, parent) in zip(contours, hiearchy[0]):
            c.next = contours[nxt] if nxt != -1 else None
            c.prev = contours[prev] if prev != -1 else None
            c.child = contours[child] if child != -1 else None
            c.parent = contours[parent] if parent != -1 else None
        return contours

    def draw_contour(self, contour, *args, **kwargs):
        return self.draw_contours([contour], 0, *args, **kwargs)

    def draw_contours(self, contours, index=-1, color=(0,255,0), thickness=1, replace=False):
        img = self if replace else self.copy()
        cv.drawContours(img, contours, index, color, thickness)
        return img
    
    def draw_rectangle(self, rect, color=(0,255,0), thickness=1, replace=False):
        img = self if replace else self.copy()
        cv.rectangle(img, rect.tl, rect.br, color, thickness)
        return img
    
    def draw_rectangles(self, rectangles, color=(0,255,0), thickness=1, replace=False):
        img = self if replace else self.copy()
        for rect in rectangles:
            cv.rectangle(img, rect.tl, rect.br, color, thickness)
        return img
    
    def write(self, filename):
        cv.imwrite(filename, self)
        return self
    
    def __repr__(self,*args):
        if(len(self.shape) == 3 and self.shape[2] == 3):
            plt.imshow(self[:,:,::-1])
            return '<Frame {}>'.format(self.shape)
        elif(len(self.shape) == 2):
            plt.imshow(self, cmap='gray')
            return '<Frame {}>'.format(self.shape)
        return super(self).__repr__(self,*args)

class VideoReader(cv.VideoCapture):
    def __init__(self, filename, _slice=slice(None, None, None)):
        self.filename = filename
        self.slice = _slice
        super().__init__(filename)
    
    # pos_msec
    # pos_frames
    # pos_avi_ratio
    # frame_width
    # frame_height
    # frame_count
    # fps
    # fourcc
    # bitrate
        
    def __setattr__(self, key, value):
        prop_name = 'CAP_PROP_'+key.upper()
        if prop_name in cv.__dict__:
            return self.set(getattr(cv, prop_name), value)
        return super().__setattr__(key, value)
    
    def __getattribute__(self, key):
        prop_name = 'CAP_PROP_'+key.upper()
        if prop_name in cv.__dict__:
            return self.get(getattr(cv, prop_name))
        return super().__getattribute__(key)
    
    def __getitem__(self, key):
        if isinstance(key, slice):
            return VideoReader(self.filename, self.slice)
        else:
            if self.pos == self.length:
                self.open(self.filename)
            assert key in range(self.length)
            if key != self.pos:
                print('jumping from {} to {}'.format(self.pos, key))
                self.pos = key
            ret, frame = vid.read()
            return frame.view(Frame)
        
    
    def __iter__(self):
        for i in range(self.length)[self.slice]:
            yield self[i]        

    @property
    def pos(self):
        return int(self.get(cv.CAP_PROP_POS_FRAMES))
    
    @pos.setter
    def pos(self, i):
        self.set(cv.CAP_PROP_POS_FRAMES, i)
    
    @property
    def width(self):
        return int(self.get(cv.CAP_PROP_FRAME_WIDTH))
    
    @property
    def height(self):
        return int(self.get(cv.CAP_PROP_FRAME_HEIGHT))
    
    @property
    def length(self):
        return int(self.get(cv.CAP_PROP_FRAME_COUNT))
    
    @property
    def size(self):
        return (self.width, self.height)


class VideoWriter(cv.VideoWriter):
    def __init__(self, filename, fourcc, fps, size):
        self.filename = filename
        if type(fourcc) == str:
            fourcc = cv.VideoWriter_fourcc(*fourcc.upper())
        super().__init__(filename, fourcc, fps, size)
    
    def __enter__(self):
        assert self.isOpened()
        return self
    
    def __exit__(self, *args):
        self.release()

def group_shapes(rectangles, matches):
    groups = [[rectangles[0]]]    
    
    for rect in rectangles[1:]:
        matching = []
        for group in groups:
            for other in group:
                if matches(rect, other):
                    matching.append(group)
                    break
        if len(matching) == 0:
            groups.append([rect])
        else:
            matching[0].append(rect)
            for other_match in matching[1:]:
                matching[0] += other_match
                del other_match[:]
    
    return sorted(groups, key=len)[::-1]



In [3]:
cap = cv.VideoCapture('~/Downloads/1,000 SPEEDRUNS IN A ROW DAY 2 _ !1000.mp4')

In [30]:
# session = livestreamer.Livestreamer()
# session.set_option('http-header', 'http-header=Client-ID=q6batx0epp608isickayubi39itsckt')
# session.set_option('twitch-oauth-token', 'oauth:6dpepteonmpvhb7h0drurw5d9s98f9')
# streams = session.streams("http://www.twitch.tv/couriway")

In [4]:
streams = streamlink.streams('https://www.twitch.tv/couriway')

In [5]:
stream = streams['best']

In [7]:
vid = VideoReader(stream.url)
with VideoWriter('tests/videos/live.3.mp4', fourcc='H264', fps=vid.fps, size=vid.size) as out:
    for frame in vid:
        out.write(frame)
vid.release()

In [8]:
vid = VideoReader('https://www.twitch.tv/videos/1098650103')

(0, 0)

In [19]:
fname = 'live.mpg'
vid_file = open(fname,'wb')

In [20]:
fd = stream.open()

In [21]:
for i in range(0,1*2048):
    if i%256==0:
        print('Buffering...')
    new_bytes = fd.read(1024)
    vid_file.write(new_bytes)

Buffering...
Buffering...
Buffering...
Buffering...
Buffering...
Buffering...
Buffering...
Buffering...


In [11]:
cap = cv.VideoCapture(fname)

In [13]:
ret, frame = cap.read()

In [15]:
cv.imshow('live_img', frame)

In [22]:
vid_file.close()

In [23]:
fd.close()

In [25]:
type(cap)

cv2.VideoCapture

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'get',
 'getBackendName',
 'getExceptionMode',
 'grab',
 'isOpened',
 'open',
 'read',
 'release',
 'retrieve',
 'set',
 'setExceptionMode']